In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)


In [ ]:
adata = sc.read_h5ad("/work/HTAP_preprocess/HTAPP_997_scRNA.h5ad")

In [ ]:
adata

In [ ]:
adata_raw = adata.raw.to_adata()

In [ ]:
import os

results_dir = "HTAP_preprocess/Final_version_Unnormalized"

# Create the directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# Now write the file
results_file = os.path.join(results_dir,"HTAPP_997_processed_raw_FINAL.h5ad")
adata_raw.write(results_file)

In [ ]:
adata_raw

In [ ]:
# Filtering low quality cells

# mitochondrial genes
adata_raw.var["mt"] = adata_raw.var_names.str.startswith("mt-")
# ribosomal genes
adata_raw.var["ribo"] = adata_raw.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata_raw.var["hb"] = adata_raw.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
sc.pp.calculate_qc_metrics(
    adata_raw, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)
adata_raw

In [ ]:
adata_raw.var_names_make_unique()

In [ ]:
sc.pl.highest_expr_genes(adata_raw, n_top=20)

In [ ]:
print("Min:", adata_raw.X.min(), "Max:", adata_raw.X.max(), "Mean:", adata_raw.X.mean())

In [ ]:
sc.pp.filter_cells(adata_raw, min_genes=200)
sc.pp.filter_genes(adata_raw, min_cells=3)

In [ ]:
adata_raw.X.sum(0)

In [ ]:
adata_raw = adata_raw[adata_raw.obs.n_genes_by_counts < 4300, :] # Filter out cells with high number of genes
adata_raw = adata_raw[adata_raw.obs.pct_counts_mt < 10, :].copy()

In [ ]:
adata_raw

In [ ]:
# Generate scatter plot and return axis object
ax = sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", show=False)

# Add a red horizontal line at y = 2500
ax.axhline(y=4300, color='red', linestyle='--', linewidth=1.5)

# Show the modified plot
plt.show()

In [ ]:
adata_raw.write(results_file)

In [ ]:
adata_normalized = adata_raw.copy()

In [ ]:
sc.pp.normalize_total(adata_normalized, target_sum=1e4)
sc.pp.log1p(adata_normalized)

In [ ]:
import os

results_dir = "HTAP_preprocess/Final_version_normalized"

# Create the directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)

# Now write the file
results_file = os.path.join(results_dir,"HTAPP_997_processed_normalized_FINAL.h5ad")
adata_normalized.write(results_file)

In [ ]:
adata_normalized.write(results_file)

In [ ]:
print("Min:", adata_raw.X.min(), "Max:", adata_raw.X.max(), "Mean:", adata_raw.X.mean())

In [ ]:
print("Min:", adata_normalized.X.min(), "Max:", adata_normalized.X.max(), "Mean:", adata_normalized.X.mean())

In [ ]:
sc.read("/work/HTAP_preprocess/HTAP_preprocess/Final_version_normalized/HTAPP_997_processed_normalized_FINAL.h5ad")

In [ ]:
sc.read("/work/HTAP_preprocess/HTAP_preprocess/Final_version_Unnormalized/HTAPP_997_processed_raw_FINAL.h5ad")